### Preprocessing 

In [1]:
import os
import cv2
import xml.etree.ElementTree as ET

def convert_to_yolo_format(bbox, img_size, class_id=0):
    """
    Convert bbox from Pascal VOC format [xmin, ymin, xmax, ymax] to YOLO format.
    """
    dw = 1. / img_size[0]
    dh = 1. / img_size[1]
    x = (bbox[0] + bbox[2]) / 2.0
    y = (bbox[1] + bbox[3]) / 2.0
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return f"{class_id} {x:.6f} {y:.6f} {w:.6f} {h:.6f}"

def parse_annotation(xml_file, image_folder):
    """
    Parse the XML file to get the bounding box coordinates and write them in YOLO format.
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    filename = root.find('filename').text
    image_path = os.path.join(image_folder, filename)
    img_size = cv2.imread(image_path).shape
    
    boxes = []
    for obj in root.iter('object'):
        class_id = 0  # TODO: Set the correct class ID if you have multiple classes
        xmlbox = obj.find('bndbox')
        bbox = (
            int(xmlbox.find('xmin').text),
            int(xmlbox.find('ymin').text),
            int(xmlbox.find('xmax').text),
            int(xmlbox.find('ymax').text),
        )
        boxes.append(convert_to_yolo_format(bbox, img_size, class_id))
    return filename, boxes

def save_yolo_annotations(annotations_path, image_folder, output_path):
    for xml_file in os.listdir(annotations_path):
        if xml_file.endswith('.xml'):
            xml_path = os.path.join(annotations_path, xml_file)
            filename, boxes = parse_annotation(xml_path, image_folder)

            # Remove image extension and add .txt extension
            base_filename = os.path.splitext(filename)[0]
            output_txt_path = os.path.join(output_path, f"{base_filename}.txt")

            # Write the YOLO formatted bounding boxes to a .txt file
            with open(output_txt_path, 'w') as file:
                for box in boxes:
                    file.write(box + "\n")


# Paths to your dataset
annotations_path = 'archive/annotations'
image_folder = 'archive/images'
output_path = 'archive/processed_images'

# Convert and save annotations in YOLO format
save_yolo_annotations(annotations_path, image_folder, output_path)


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [38]:
import os
import cv2
import torch
import numpy as np
from easyocr import Reader


# Step 2: Detect license plates in images using a pre-trained YOLO model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

def detect_license_plates(image_directory):
    images, labels, cropped_plates = [], [], []
    reader = Reader(['en'])

    for filename in os.listdir(image_directory):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(image_directory, filename)
            img = cv2.imread(img_path)
            results = model(img)
            detections = results.xyxy[0]

            for *xyxy, conf, cls in detections:
                x_min, y_min, x_max, y_max = map(int, xyxy)
                cropped_plate = img[y_min:y_max, x_min:x_max]
                cropped_plates.append(cropped_plate)

                # Step 3: Optional - Use EasyOCR to recognize text from cropped license plates
                ocr_result = reader.readtext(cropped_plate)
                label = max(ocr_result, key=lambda result: result[2])[1] if ocr_result else 'N/A'
                labels.append(label)

            images.append(img)

    return images, labels, cropped_plates

# Assume you have a directory of images to process
image_directory = 'archive/images'
images, labels, cropped_plates = detect_license_plates(image_directory)



requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'wheel>=0.38.0'] not found, attempting AutoUpdate...


Using cache found in /Users/prayusishah/.cache/torch/hub/ultralytics_yolov5_master



requirements: AutoUpdate success ✅ 5.0s, installed 3 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'wheel>=0.38.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-4-8 Python-3.12.1 torch-2.2.2 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [42]:

batch_size = 32
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, batch_size=batch_size)

ValueError: Found input variables with inconsistent numbers of samples: [433, 1431]

In [40]:
python val.py --weights path/to/your/model.pt --data data.yaml --img 640

SyntaxError: invalid syntax (511807831.py, line 1)